### GradientAscent Solutions 

### Course Scheduling at the Marshall School of Business

The proposed MIP optimization routine seeks to aid Shannon and Hal in Phase-1 for allotting departments to classroom-sessions. The optimization maximizes cumulative department preference while minimizing inter-department cumulative preference score disparity.

The required resource (classroom-timeslots) for each department is organized into groups on the basis of section-size (Small, Medium or Large), and the number of credits (1.5 or 3). The required number of timeslots for each department against each such group is set as a hard constraint.

Preferences: Historical data is given precedence, i.e. if a Department has been offered the same Classroom-Day-Timeslot over a period of time it is assumed that such an assignment is favorable to the department.

#### Definitions:

$$\begin{aligned}
x_{ijk} = \text{Binary Decision Variable; Allot Dept 'i' to Classroom-Timeslot 'k' on Day-Index 'j'} \\
p_{ijk} = \text{Preference Data; Dept 'i' for Classroom-Timeslot 'k' on Day-Index 'j'} \\
U_{DP} = \text{Maximum: Weighted Cumulative Department Preference} \\ 
L_{DP} = \text{Minimum: Weighted Cumulative Department Preference}\\
q_{i} = \text{Total required timeslots over all classrooms and 1.5 / 3 credit sections} \\
q_{Sj_{1.5}} = \text{Number of timeslots required by Department 'i' for "Small" sections worth 1.5 credits} \\
q_{Sj_{3.0}} = \text{Number of timeslots required by Department 'i' for "Small" sections worth 3.0 credits} \\
q_{Mj_{1.5}} = \text{Number of timeslots required by Department 'i' for "Medium" sections worth 1.5 credits} \\
q_{Mj_{3.0}} = \text{Number of timeslots required by Department 'i' for "Medium" sections worth 3.0 credits} \\
q_{Lj_{1.5}} = \text{Number of timeslots required by Department 'i' for "Large" sections worth 1.5 credits} \\
q_{Lj_{3.0}} = \text{Number of timeslots required by Department 'i' for "Large" sections worth 3.0 credits} \\
\lambda = \text{Tuning Parameter for Model}
\end{aligned}$$

### LP Formulation

$$\begin{aligned}
\textbf{maximize: } && \displaystyle \sum_{i \in I}\sum_{j \in J}\sum_{k \in K} x_{ijk}p_{ijk} - \lambda(U_{DP} - L_{DP}) \\
\text{where} && U_{DP} \ge \displaystyle \frac{(\sum_{j \in J}\sum_{k \in K} x_{ijk}p_{ijk})}{q_{i}}\\
&& L_{DP} \le \displaystyle \frac{(\sum_{j \in J}\sum_{k \in K} x_{ijk}p_{ijk})}{q_{i}} \\
\textbf{subject to: } \\
\text{Binary DV} && x_{ijk} \in \{0, 1\} \\
\text{One Dept maps to One Classroom-Timeslot} && \sum_{i \in I} x_{ijk} \le 1 \ \forall \  j \in J, \ k \in K \\
\text{Small Section, 1.5 Credit Timeslots} && \sum_{k \in S}\sum_{j \in J_{1.5}} x_{ijk} \ge q_{Sj_{1.5}i} \ \forall \ i \in I \\
\text{Small Section, 3.0 Credit Timeslots} && \sum_{k \in S}\sum_{j \in J_{3.0}} x_{ijk} \ge q_{Sj_{3.0}i} \ \forall \ i \in I \\
\text{Medium Section, 1.5 Credit Timeslots} && \sum_{k \in M}\sum_{j \in J_{1.5}} x_{ijk} \ge q_{Mj_{1.5}i} \ \forall \ i \in I \\
\text{Medium Section, 3.0 Credit Timeslots} && \sum_{k \in M}\sum_{j \in J_{3.0}} x_{ijk} \ge q_{Mj_{3.0}i} \ \forall \ i \in I \\
\text{Large Section, 1.5 Credit Timeslots} && \sum_{k \in L}\sum_{j \in J_{1.5}} x_{ijk} \ge q_{Lj_{1.5}i} \ \forall \ i \in I \\
\text{Large Section, 3.0 Credit Timeslots} && \sum_{k \in L}\sum_{j \in J_{3.0}} x_{ijk} \ge q_{Lj_{3.0}i} \ \forall \ i \in I \\
\text{Pref > Allotment} && \ p_{ijk} \ge x_{ijk} \ \forall i \in I, \ j \in J, \ k \in K \\
\text{IntraDept: Conflicting 1.5 and 3-Credit Courses} && x_{iMk} + x_{iMWk} \le 1 \ \forall i \in I, \ k \in K \\
&& x_{iWk} + x_{iMWk} \le 1 \ \forall i \in I, \ k \in K \\
&& x_{iTk} + x_{iTHk} \le 1 \ \forall i \in I, \ k \in K \\
&& x_{iHk} + x_{iTHk} \le 1 \ \forall i \in I, \ k \in K \\
\text{InterDept: Conflicting 1.5 and 3-Credit Courses} && x_{i^*Mk} + x_{iMWk} \le 1 \ \forall i \in I, \ i^* \in I - \{i\}, \ k \in K \\
&& x_{i^*Wk} + x_{iMWk} \le 1 \ \forall i \in I, \ i^* \in I - \{i\}, \ k \in K \\
&& x_{i^*Tk} + x_{iTHk} \le 1 \ \forall i \in I, \ i^* \in I - \{i\}, \ k \in K \\
&& x_{i^*Hk} + x_{iTHk} \le 1 \ \forall i \in I, \ i^* \in I - \{i\}, \ k \in K \\
\end{aligned}$$

In [2]:
import numpy as np
import pandas as pd
import gurobipy as grb

course_sizes = pd.read_excel('Final_TOYDATA.xlsx', 'Course_SizeCredit_Breakdown')
classroom_sizes = pd.read_excel('Final_TOYDATA.xlsx', 'Classrooms')
prefs_D1  = pd.read_excel('Final_TOYDATA.xlsx', 'HistoricalPref_D1')
prefs_D2  = pd.read_excel('Final_TOYDATA.xlsx', 'HistoricalPref_D2')
prefs_D3  = pd.read_excel('Final_TOYDATA.xlsx', 'HistoricalPref_D3')

K = list(prefs_D1.index)

course_sizes

,D1,D2,D3
Total_Courses,10,7,9
Small_Cred1.5,4,2,0
Small_Cred3,1,3,3
Med_Cred1.5,0,0,3
Med_Cred3,1,0,1
Large_Cred1.5,1,1,1
Large_Cred3,3,1,1


In [3]:
prefs = {}
prefs['D1'] = prefs_D1
prefs['D2'] = prefs_D2
prefs['D3'] = prefs_D3

prefs['D1']

,M,T,W,H,MW,TH
C1-TS1,2,2,1,1,2,2
C1-TS2,2,1,2,0,2,2
C1-TS3,0,1,2,2,1,1
C2-TS1,0,1,0,0,2,1
C2-TS2,1,0,2,0,2,1
C2-TS3,1,1,0,0,0,1
C3-TS1,1,2,1,1,2,1
C3-TS2,2,0,2,0,1,0
C3-TS3,2,2,1,0,2,2
C4-TS1,2,2,1,1,1,0


In [4]:
session_groups = list(course_sizes.index)
I = list(course_sizes.columns) ##All Departments
J = ['M', 'T', 'W', 'H', 'MW', 'TH'] ##All Day Indices 
J1p5 = J[0:4] ## Day Indices for 1.5 credit courses
J3p0 = J[4:] ## Day Indices for 3.0 credit courses

# All Small, Medium and Large Classroom Sessions

S = []
M = []
L = []

classroom_sizes

,Type
C1,S
C2,M
C3,L
C4,S
C5,L


In [5]:
for k in K:
    if 'C1' in k or 'C4' in k:
        S.append(k)
    elif 'C2' in k:
        M.append(k)
    elif 'C3' in k or 'C5' in k:
        L.append(k)


In [6]:
mod = grb.Model()
mod.setParam('TimeLimit', 1*60)
## Decision Variables
x = {}
for i in I:
    for j in J:
        for k in K:
            x[i,j,k] = mod.addVar(vtype = grb.GRB.BINARY, name = 'X{0}{1}{2}'.format(i,j,k))

Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


In [7]:
## Auxiliary Decision Variables:

U_dpmax = mod.addVar(lb = 0, name = 'DeptPrefs_UpperBound')
L_dpmin = mod.addVar(lb = 0, name = 'DeptPrefs_LowerBound')

for i in I:
    mod.addConstr(U_dpmax * course_sizes.loc['Total_Courses', i] >= sum(x[i,j,k]*prefs[i].loc[k, j] for j in J for k in K))

for i in I:
    mod.addConstr(L_dpmin * course_sizes.loc['Total_Courses', i] <= sum(x[i,j,k]*prefs[i].loc[k, j] for j in J for k in K))
    

## Preference Constraints:
for i in I:
    for j in J:
        for k in K:
            mod.addConstr(x[i,j,k] <= prefs[i].loc[k,j])


In [8]:
## Constraints:

#Every day - classroom session combination must be given only to ONE department
for j in J:
    for k in K:
        mod.addConstr(sum(x[i,j,k] for i in I) <= 1)
        
#Every department must be given required number of timeslots for Small sections of 1.5 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J1p5 for k in S) >= course_sizes.loc['Small_Cred1.5', i], 
                 name = 'Small Section 1.5 Credit Sufficiency for {0}'.format(i))

#Every department must be given required number of timeslots for Small sections of 3.0 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J3p0 for k in S) >= course_sizes.loc['Small_Cred3', i], 
                 name = 'Small Section 3.0 Credit Sufficiency for {0}'.format(i))

#Every department must be given required number of timeslots for Medium sections of 1.5 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J1p5 for k in M) >= course_sizes.loc['Med_Cred1.5', i], 
                 name = 'Med Section 1.5 Credit Sufficiency for {0}'.format(i))
    
#Every department must be given required number of timeslots for Medium sections of 3.0 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J3p0 for k in M) >= course_sizes.loc['Med_Cred3', i], 
                 name = 'Med Section 3.0 Credit Sufficiency for {0}'.format(i))

#Every department must be given required number of timeslots for Large sections of 1.5 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J1p5 for k in L) >= course_sizes.loc['Large_Cred1.5', i], 
                 name = 'Large Section 1.5 Credit Sufficiency for {0}'.format(i))
    
#Every department must be given required number of timeslots for Large sections of 3.0 credits

for i in I:
    mod.addConstr(sum(x[i,j,k] for j in J3p0 for k in L) >= course_sizes.loc['Large_Cred3', i], 
                 name = 'Large Section 3.0 Credit Sufficiency for {0}'.format(i))

In [10]:
## 1.5 Credit and 3-Credit Conflict Resolution Constraints:

#IntraDept
for i in I:
    for k in K:
        mod.addConstr(x[i, 'M', k] + x[i, 'MW', k] <= 1)
        mod.addConstr(x[i, 'W', k] + x[i, 'MW', k] <= 1)

for i in I:
    for k in K:
        mod.addConstr(x[i, 'T', k] + x[i, 'TH', k] <= 1)
        mod.addConstr(x[i, 'H', k] + x[i, 'TH', k] <= 1)

#InterDept
for i in I:
    I_copy = I[:]
    I_star = I_copy
    I_star.remove(i)
    for i_ in I_star:
        for k in K:
            mod.addConstr(x[i_,'M',k] + x[i, 'MW', k] <= 1)
            mod.addConstr(x[i_,'W',k] + x[i, 'MW', k] <= 1)
            mod.addConstr(x[i_,'T',k] + x[i, 'TH', k] <= 1)
            mod.addConstr(x[i_,'H',k] + x[i, 'TH', k] <= 1)
            

In [11]:
## Setting Objective

alpha = 100 ##Scaling Parameter

mod.setObjective(sum(x[i,j,k]*prefs[i].loc[k,j] for i in I for j in J for k in K) - alpha*(U_dpmax - L_dpmin), 
                 sense = grb.GRB.MAXIMIZE)

In [12]:
mod.optimize()

Optimize a model with 1104 rows, 272 columns and 2656 nonzeros
Variable types: 2 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -101.0000048
Presolve removed 1025 rows and 73 columns
Presolve time: 0.02s
Presolved: 79 rows, 199 columns, 832 nonzeros
Variable types: 2 continuous, 197 integer (197 binary)

Root relaxation: objective 8.700000e+01, 154 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   87.00000    0    4 -101.00000   87.00000   186%     -    0s
H    0     0                      66.3650794   87.00000  31.1%     -    0s
H    0     0                      75.0634921   87.00000  15.9%     -    0s
     0     0   87.00000    0   16   75.06349   87.00000  15.9

In [199]:
var_vals = {}
for var in mod.getVars():
    var_vals[var.VarName] = var.x 

aux_vars = list(var_vals.keys())[270:]

aux_var_vals = {}

for a in aux_vars:
    aux_var_vals[a] = var_vals[a]
    var_vals.pop(a)
    
aux_var_vals

{'DeptPrefs_LowerBound': 3.2857142857142874,
 'DeptPrefs_UpperBound': 3.3333333306032764}

In [200]:
alloc_dict = {}

alloc_dict['D1'] = {}
alloc_dict['D2'] = {}
alloc_dict['D3'] = {}

alloc_dict

{'D1': {}, 'D2': {}, 'D3': {}}

In [201]:
for i in I:
    for key in var_vals.keys():
        if i in key:
            alloc_dict[i].update({key : var_vals[key]})
        else:
            continue

In [202]:
d1 = pd.Series(alloc_dict['D1'])
d1 = pd.DataFrame(d1)
d1['D1_Index'] = d1.index
d1.index = range(0,90)
d1.columns = ['D1_Allot', 'D1_Index']

d2 = pd.Series(alloc_dict['D2'])
d2 = pd.DataFrame(d2)
d2['D2_Index'] = d2.index
d2.index = range(0,90)
d2.columns = ['D2_Allot', 'D2_Index']

d3 = pd.Series(alloc_dict['D3'])
d3 = pd.DataFrame(d3)
d3['D3_Index'] = d3.index
d3.index = range(0,90)
d3.columns = ['D3_Allot', 'D3_Index']


In [203]:
frames = [d1, d2, d3]
dept_alls = pd.concat(frames, axis = 1)

In [205]:
dept_alls.to_csv('DeptAllocations.csv')

~END OF DOCUMENT~

In [11]:
prefs['D1'].loc['C1-TS1','M']

2

In [9]:
J

['M', 'T', 'W', 'H', 'MW', 'TH']